In [ ]:
from ultralytics import YOLO

model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# ✅ Correct training call
model.train(
    data='/home/pegasus/Documents/NUMBERplates/My First Project.v12i.yolov8W/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    patience=5,
    save=True,
    name='retrained_model',
    resume=False
)


In [ ]:


# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Directory containing test images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"
output_txt = "ocr_results_simple.txt"

# Create output file
with open(output_txt, "w") as f:
    for image_file in os.listdir(image_folder):
        if not image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        image_path = os.path.join(image_folder, image_file)

        # Run YOLO prediction
        results = model.predict(image_path, imgsz=640, conf=0.3)

        # Load original image
        img = cv2.imread(image_path)

        for r in results:
            for i, box in enumerate(r.boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                plate_img = img[y1:y2, x1:x2]

                # Resize and basic grayscale conversion
                plate_img = cv2.resize(plate_img, (400, 100))
                gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)

                # Basic threshold to help Tesseract
                _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

                # Run Tesseract OCR
                config = "--psm 7"
                text = pytesseract.image_to_string(thresh, config=config).strip()

                # Log results
                f.write(f"{image_file} - Plate {i+1}: {text}\n")
                print(f"{image_file} - Plate {i+1}: {text}")


In [ ]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Folder with input images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"

# Loop through all images
for image_file in os.listdir(image_folder):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    image_path = os.path.join(image_folder, image_file)

    # Run YOLO prediction
    results = model.predict(image_path, imgsz=640, conf=0.3)

    # Load image with OpenCV, convert to RGB
    cv_img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

    # Convert to PIL image for drawing
    pil_img = Image.fromarray(rgb_img)
    draw = ImageDraw.Draw(pil_img)

    # Draw all detected bounding boxes
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=3)

    # Show image with bounding boxes
    pil_img.show()


In [ ]:
## saving cropped images
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Input folder with test images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"

# Output folder for cropped number plates
output_folder = "/home/pegasus/Documents/NUMBERplates/cropped_plates"
os.makedirs(output_folder, exist_ok=True)

# Iterate over images
for image_file in os.listdir(image_folder):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    image_path = os.path.join(image_folder, image_file)
    results = model.predict(image_path, imgsz=640, conf=0.3)

    # Load image using OpenCV
    img = cv2.imread(image_path)

    for r_index, r in enumerate(results):
        for i, box in enumerate(r.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cropped = img[y1:y2, x1:x2]

            # Save cropped plate image
            cropped_filename = f"{os.path.splitext(image_file)[0]}_plate{i+1}.jpg"
            cropped_path = os.path.join(output_folder, cropped_filename)
            cv2.imwrite(cropped_path, cropped)

print(f"✅ Cropped plates saved to: {output_folder}")


OCR

In [ ]:
from paddleocr import PaddleOCR
import os

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Path to your folder of plate images
image_folder = '/home/pegasus/Documents/NUMBERplates/cropped_plates'
output_txt_path = '/home/pegasus/Documents/NUMBERplates/ocr_results.txt'

# Get image list
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Open the output file
with open(output_txt_path, 'w', encoding='utf-8') as file:
    # Iterate through each image and extract text
    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)
        result = ocr.ocr(image_path, cls=True)

        plate_text = []
        if result and result[0]:
            for line in result[0]:
                plate_text.append(line[1][0])

        # Join detected text into a single string
        final_text = ' '.join(plate_text).strip()
        final_output = final_text if final_text else 'No text detected'

        # Print to console and write to file
        print(f"{image_name} -> {final_output}")
        file.write(f"{image_name} -> {final_output}\n")

print(f"\n✅ All OCR results saved to: {output_txt_path}")


In [ ]:
from paddleocr import PaddleOCR
import cv2
import os

# Paths
image_folder = '/home/pegasus/Documents/NUMBERplates/cropped_plates_for_ocr'  # Replace with your folder path
output_txt = 'results.txt'

# Initialize OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Create/overwrite result file
with open(output_txt, 'w') as f_out:
    # List all images
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)

        # Load and preprocess
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        contrast = clahe.apply(gray)

        blur = cv2.GaussianBlur(contrast, (3, 3), 0)
        _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Save temporary image for OCR
        temp_path = 'temp_preprocessed.jpg'
        cv2.imwrite(temp_path, thresh)

        # OCR
        result = ocr.ocr(temp_path, cls=True)
        plate_text = []
        if result and result[0]:
            for line in result[0]:
                plate_text.append(line[1][0])

        final_text = ' '.join(plate_text) if plate_text else 'No text detected'
        print(f"{image_name} -> {final_text}")

        # Save to txt file
        f_out.write(f"{image_name} -> {final_text}\n")

print(f"\n✅ All results saved to: {output_txt}")


## full script

In [ ]:
#-------GUI
import os
import cv2
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
from ultralytics import YOLO
from paddleocr import PaddleOCR
from datetime import datetime

# Initialize OCR and YOLO model
ocr = PaddleOCR(use_angle_cls=True, lang='en')
model = YOLO("/home/pegasus/Documents/NUMBERplates/best2.pt")

# GUI to select multiple images and output filename
def select_images():
    return filedialog.askopenfilenames(title="Select Images", filetypes=[("Image Files", "*.png *.jpg *.jpeg")])

def get_output_filename():
    return simpledialog.askstring("Output File", "Enter output text filename (e.g., result.txt):")

def process_images(image_paths, output_filename):
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    output_text_path = os.path.join(output_dir, output_filename)
    all_results = []

    for image_path in image_paths:
        cropped_texts = []
        image_name = os.path.basename(image_path)
        results = model.predict(image_path, imgsz=320, conf=0.2)
        orig_img = cv2.imread(image_path)

        for i, r in enumerate(results):
            boxes = r.boxes.xyxy.cpu().numpy().astype(int)

            for j, box in enumerate(boxes):
                x1, y1, x2, y2 = box
                crop = orig_img[y1:y2, x1:x2]

                # Save cropped image
                crop_name = f"crop_{os.path.splitext(image_name)[0]}_{j}.jpg"
                crop_path = os.path.join(output_dir, crop_name)
                cv2.imwrite(crop_path, crop)

                # OCR
                result = ocr.ocr(crop_path, cls=True)
                plate_text = []

                if result and result[0]:
                    for line in result[0]:
                        text = line[1][0]
                        cleaned = ''.join(c for c in text if c.isalnum()).upper()
                        plate_text.append(cleaned)
                else:
                    plate_text.append("NoText")

                plate_line = ' '.join(plate_text)
                cropped_texts.append(plate_line)

                # Draw bounding box
                cv2.rectangle(orig_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Save annotated image
        annotated_path = os.path.join(output_dir, f"annotated_{image_name}")
        cv2.imwrite(annotated_path, orig_img)

        # Collect all plate lines for this image
        all_results.append(f"{image_name} -> {' '.join(cropped_texts)}")
        print(f"{image_name} Detected Plate: {' '.join(cropped_texts)}")

    # Save to output text file
    with open(output_text_path, 'w') as f:
        for line in all_results:
            f.write(line + "\n")

    messagebox.showinfo("Done", f"Processed {len(image_paths)} image(s).\nText saved to {output_text_path}.")

# Main GUI window
root = tk.Tk()
root.withdraw()  

# Run workflow
images = select_images()
if images:
    output_name = get_output_filename()
    if output_name:
        process_images(images, output_name if output_name.endswith(".txt") else output_name + ".txt")
    else:
        print("❌ Output filename not provided.")
else:
    print("❌ No images selected.")
